In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from models import GNNClassifier
import torch
import numpy as np
import util
import scipy
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
import os

In [25]:
random_state = 1
batch_size = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(random_state)
# torch.cuda.manual_seed(random_state)
np.random.seed(random_state)

path = os.path.join('./', 'data', 'TU')
dataset = TUDataset(path, name='MUTAG').shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

/Users/k/opt/anaconda3/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [57]:
from sklearn.metrics import accuracy_score

def train(model, optimizer, dataloader, device):
    model.train()

    total_loss = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = model.get_loss(data)
        loss.backward()
        total_loss += data.num_graphs * loss.item()
        optimizer.step()
        
    return total_loss / len(train_dataset)

def test(model, dataloader, device):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in dataloader:

            data = data.to(device)
            pred = model.predict(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.extend(pred)
            labels.extend(label)
    total_acc = accuracy_score(predictions, labels)
    return total_acc
    

model_config = {}
model_config["input_dim"] = 7
model_config["hidden_dim"] = 32
model_config["output_dim"] = 32
model_config["n_class"] = 2
model_config["c_u"] = 1
model_config["c_sigma"] = 2
model_config["num_layers"] = 2
learning_rate = 0.001
epochs = 100

model = GNNClassifier(model_config)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

model = model.to(device)
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, train_dataloader, device)
    train_acc = test(model, train_dataloader, device)
    test_acc = test(model, test_dataloader, device)
    print('Epoch: {:03d}, train_loss: {:.4f}, train_acc: {:.4f}, test_acc: {:.4f}'.format(epoch, train_loss, train_acc, test_acc))
    

Epoch: 001, train_loss: 0.6301, train_acc: 0.6706, test_acc: 0.6111
Epoch: 002, train_loss: 0.5770, train_acc: 0.6706, test_acc: 0.6111
Epoch: 003, train_loss: 0.5705, train_acc: 0.6706, test_acc: 0.6111
Epoch: 004, train_loss: 0.5682, train_acc: 0.6706, test_acc: 0.6111
Epoch: 005, train_loss: 0.5717, train_acc: 0.6706, test_acc: 0.6111
Epoch: 006, train_loss: 0.5643, train_acc: 0.6706, test_acc: 0.6111
Epoch: 007, train_loss: 0.5608, train_acc: 0.6706, test_acc: 0.6111
Epoch: 008, train_loss: 0.5465, train_acc: 0.6706, test_acc: 0.6111
Epoch: 009, train_loss: 0.5509, train_acc: 0.6706, test_acc: 0.6111
Epoch: 010, train_loss: 0.5388, train_acc: 0.6706, test_acc: 0.6111
Epoch: 011, train_loss: 0.5465, train_acc: 0.6706, test_acc: 0.6111
Epoch: 012, train_loss: 0.5286, train_acc: 0.6706, test_acc: 0.6111
Epoch: 013, train_loss: 0.5296, train_acc: 0.6706, test_acc: 0.6111
Epoch: 014, train_loss: 0.5217, train_acc: 0.6706, test_acc: 0.6111
Epoch: 015, train_loss: 0.5227, train_acc: 0.670

In [47]:
for name, parameters in model.named_parameters():
    print(name, ':', parameters.size())

blocks.0.fc.weight : torch.Size([32, 7])
blocks.0.fc.bias : torch.Size([32])
blocks.1.fc.weight : torch.Size([32, 32])
blocks.1.fc.bias : torch.Size([32])
mlp.weight : torch.Size([2, 32])
mlp.bias : torch.Size([2])
